In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# import surprise
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from math import sqrt

#from sklearn import datasets, linear_model
%cd /content/drive/My Drive/Colab Notebooks/Thesis

/content/drive/My Drive/Colab Notebooks/Thesis


In [0]:
ratings_file = './ml-latest-small/ratings.csv'

In [0]:
header = ['userId', 'movieId', 'rating','timestamp']
df=pd.read_csv('ml-latest-small/ratings.csv')
df = df.drop(['timestamp'],axis =1)
df.head()
# df.shape

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [0]:
n_users = df.userId.unique().shape[0]
n_movies = df.movieId.unique().shape[0]

print ('num of users = ' + str(n_users)+ '| num of movies = ' + str(n_movies))

num of users = 610| num of movies = 9724


In [0]:
df.movieId.unique().shape

(9724,)

In [0]:
## Movies ids are not consequtive nor incremental. The next cell fails because although here are 
## 9724 movies, there are ids with larger numbers, for example:
###userId        105.0
###movieId    175401.0
###rating          4.0

#movie ids need to be fixed

In [0]:
#create ids for movies
moviesids = df[['movieId']].drop_duplicates()
moviesids = moviesids.sort_values(by='movieId')

#s = pd.Series(moviesids.movieId.values.tolist())
moviesids['mid'] = range(0, len(moviesids))
moviesids['mid'] = moviesids['mid']+1
# moviesids

In [0]:
df = pd.merge(df,moviesids[['movieId','mid']],on='movieId',how='left')
# df

In [0]:
df = df.drop(columns=['movieId'])
df = df.rename(index=str,columns={'mid':'movieId'})
# df

In [0]:
# https://towardsdatascience.com/how-to-build-a-simple-recommender-system-in-python-375093c3fb7d
movie_titles = pd.read_csv('ml-latest-small/movies.csv')
# movie_titles.head()

In [0]:
df = pd.merge(df, movie_titles, on='movieId')
# df.head()

### Dataset analysis/exploration

In [0]:
ratings = pd.DataFrame(df.groupby('title')['rating'].mean())
# ratings.head()

In [0]:
ratings['number_of_ratings'] = df.groupby('title')['rating'].count()
# ratings.head()

In [0]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# ratings['rating'].hist(bins=50)

In [0]:
# ratings['number_of_ratings'].hist(bins=60)

In [0]:
# import seaborn as sns
# sns.jointplot(x='rating', y='number_of_ratings', data=ratings)


In [0]:
# #location and amount of missing values
# missing_df = pd.DataFrame(df.isnull().sum()).T
# missing_df

In [0]:
# create utility matrix
movie_matrix = df.pivot_table(index='userId', columns='title', values='rating').fillna(0)
# movie_matrix.head()

In [0]:
# #location and amount of missing values
# missing_df = pd.DataFrame(movie_matrix.isnull().sum()).T
# missing_df

In [0]:
# ratings.sort_values('number_of_ratings', ascending=False).head(10)

In [0]:
# specialist_user_rating = movie_matrix['Specialist, The (1994)']
# specialist_user_rating.head()

In [0]:
# corr_movie = pd.DataFrame(similar_to_currentmovie, columns=['Correlation'])
# corr_movie.dropna(inplace=True)
# # corr_movie.head()

# corr_movie = corr_movie.join(ratings['number_of_ratings'])
# corr_movie.head()

## cross validation train test split

In [0]:
train_data, test_data = train_test_split(movie_matrix,test_size=0.25)

In [0]:
train_data.shape

(457, 5401)

In [0]:
test_data.shape


(153, 5401)

###  Top K - KNN Predictions

In [0]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine',n_neighbors=10, algorithm = 'brute')
model_knn.fit(movie_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [0]:
# # https://towardsdatascience.com/how-did-we-build-book-recommender-systems-in-an-hour-part-2-k-nearest-neighbors-and-matrix-c04b3c2ef55c
# query_index = np.random.choice(movie_matrix.shape[0])
# distances, indices = model_knn.kneighbors(movie_matrix.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 10)

# for i in range(0, len(distances.flatten())):
#     if i == 0:
#         print('Recommendations for {0}:\n'.format(movie_matrix.index[query_index]))
#     else:
#         print('{0}: {1}, with distance of {2}:'.format(i, movie_matrix.index[indices.flatten()[i]], distances.flatten()[i]))

### User- and Item-based CF

In [0]:
def fast_similarity(ratings, kind='user', epsilon=1e-9):
    # epsilon -> small number for handling dived-by-zero errors
    if kind == 'user':
        sim = ratings.dot(ratings.T) + epsilon
    elif kind == 'item':
        sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [0]:
#the respective matrix must be produced for items
similarity_matrix_users = fast_similarity(movie_matrix,kind='user')

In [0]:
similarity_matrix_items = fast_similarity(movie_matrix,kind='item')

In [0]:
#source: https://www.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/
def predict_topk(ratings, similarity1, kind='user', k=40):
    # start_pred = time.time()
    col_names = ratings.columns.tolist()
    user_names = ratings.index.to_list()
    similarity = similarity1.to_numpy()
    pred = np.zeros(ratings.shape)
    k=k+1
    if kind == 'user':
        for i in range(ratings.shape[0]):
            top_k_users = [np.argsort(similarity[:,i])[:-k-1:-1]]
            for j in range(ratings.shape[1]):
                ################
                pred[i, j] = similarity[i, :][top_k_users].dot(ratings.to_numpy()[:, j][top_k_users]) 
                ## The command above did not work. The solution below however, takes a lot of time to complete
                ## If it takes too long on your laptop, you should fix this
                # jname = col_names[j]
                # dotar = np.zeros(k)
                # c=0
                # for us in top_k_users[0]:
                #     val = ratings.loc[user_names[us],jname]
                #     dotar[c]=val
                #     c+=1
                # ## validate?
                # ################
                # pred[i, j] = similarity[i, :][top_k_users].dot(dotar) 
                pred[i, j] /= np.sum(np.abs(similarity[i, :][top_k_users]))

                # print("user_time_prediction",time.time()-start_pred) 
    if kind == 'item':
        for j in range(ratings.shape[1]):
            top_k_items = [np.argsort(similarity[:,j])[:-k-1:-1]]
            for i in range(ratings.shape[0]):
                ################
                pred[i, j] = similarity[j, :][top_k_items].dot(ratings.to_numpy()[i, :][top_k_items].T) 
                # The command above did not work. The solution below however, takes a lot of time to complete
                # If it takes too long on your laptop, you should fix this
                # dotvar = np.zeros(k)
                # c=0
                # for it in top_k_items[0]:
                #     val = ratings.loc[user_names[i],col_names[it]]
                #     dotvar[c]=val
                #     c+=1
                # ## validate?
                # ################
                # pred[i, j] = similarity[j, :][top_k_items].dot(dotvar.T) 
                
                pred[i, j] /= np.sum(np.abs(similarity[j, :][top_k_items])) 
                # print("item _time_prediction",time.time()-start_pred)       
    
    return pred

In [0]:
k = 100

In [0]:
start_pred = time.time()
predicted_values_users = predict_topk(movie_matrix,similarity_matrix_users,'user',k)
end_user_pred = time.time()-start_pred
print("user_time_prediction",end_user_pred)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


user_time_prediction 80.88796830177307


In [0]:
start=time.time()
predicted_values_items = predict_topk(movie_matrix,similarity_matrix_items,'item',k)
end_item_pred = time.time()-start
print("item _time_prediction",end_item_pred)   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


item _time_prediction 88.318195104599


In [0]:
tst = movie_matrix.copy()
tst.insert(0, 'order', range(0, 0 + len(tst)))
tst = tst[['order']]
tst = tst.reset_index()

movietst = pd.DataFrame(movie_matrix.columns)
movietst.insert(0, 'order', range(0, 0 + len(movietst)))

results_users= pd.DataFrame()

results_items = pd.DataFrame()

for index,row in test_data.iterrows():
    usr_tst = pd.DataFrame(row)
    #remove the non zero ratings of this user
    usr_tst = usr_tst[usr_tst[index]>0]
    order_of_index= tst.loc[tst['userId']==index,'order'].values.tolist()[0]
    #predicted ratings
    for index1,row1 in usr_tst.iterrows():
        order_of_index1= movietst.loc[movietst['title']==index1,'order'].values.tolist()[0]
        predicted_user = predicted_values_users[order_of_index,order_of_index1]
        real_value = row1[index]
    
        predicted_item = predicted_values_items[order_of_index,order_of_index1]
        
        results_users = results_users.append({'user':index,
                                 'item':index1,
                                 'predicted':predicted_user,
                                 'real_value':real_value},ignore_index=True)
        
        results_items = results_items.append({'user':index,
                                 'item':index1,
                                 'predicted':predicted_item,
                                 'real_value':real_value},ignore_index=True)

In [0]:
start_res = time.time()
results_users= results_users.drop_duplicates()
end_user_res = time.time()-start_res
print("user _time_prediction",end_user_res) 

results_items= results_items.drop_duplicates()
end_item_res = time.time()-start_res
print("item _time_prediction",end_item_res) 

user _time_prediction 0.016565799713134766
item _time_prediction 0.027730941772460938


##MAE

In [0]:
print('user-based CF MAE:'+str(mean_absolute_error(results_users.predicted.values.tolist(),results_users.real_value.values.tolist())))

In [0]:
print('item-based CF MAE:'+str(mean_absolute_error(results_items.predicted.values.tolist(),results_items.real_value.values.tolist())))

In [0]:
# print('user-based CF MSE:'+str(mean_squared_error(results_users.predicted.values.tolist(),results_users.real_value.values.tolist())))


In [0]:
# print('item-based CF MSE:'+str(mean_squared_error(results_items.predicted.values.tolist(),results_items.real_value.values.tolist())))

## RMSE

In [0]:
rmseval = sqrt(mean_squared_error(results_users.real_value.values.tolist(), results_users.predicted.values.tolist()))
print('user-based CF RMSE: '+ str(rmseval))

In [0]:
rmsevalitem = sqrt(mean_squared_error(results_items.real_value.values.tolist(), results_items.predicted.values.tolist()))
print('item-based CF RMSE: '+ str(rmsevalitem))

#Model-based Approaches
##Explicit Feedback 


In [0]:
!pip install surprise
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import BaselineOnly
from surprise import SVD
from surprise import NMF
from surprise import accuracy
from surprise.model_selection import KFold
from surprise import Dataset
from surprise.model_selection import train_test_split


     |████████████████████████████████| 6.5MB 6.4MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1675375 sha256=93cafe579970e086cd1989dbdaee48d71f04c149841c5080c5991cd70650c129
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [0]:
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

###SVD


https://github.com/JieniChen/Recommender-System#svd

In [0]:
# method on the given link : http://surpriselib.com and https://surprise.readthedocs.io/en/stable/getting_started.html#use-cross-validation-iterators
svd = SVD(n_factors = 50)
# Run 5-fold cross-validation and print results.
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9337  0.9326  0.9363  0.9377  0.9289  0.9338  0.0031  
MAE (testset)     0.7368  0.7331  0.7384  0.7401  0.7319  0.7360  0.0031  
Fit time          3.12    3.13    3.14    3.13    3.14    3.13    0.01    
Test time         0.17    0.24    0.17    0.24    0.16    0.20    0.03    


{'fit_time': (3.1234443187713623,
  3.1314637660980225,
  3.1416878700256348,
  3.1325795650482178,
  3.137659788131714),
 'test_mae': array([0.73680658, 0.73307713, 0.73838333, 0.74010675, 0.73186746]),
 'test_rmse': array([0.93371449, 0.93256738, 0.9363005 , 0.93772937, 0.92891926]),
 'test_time': (0.16959881782531738,
  0.23665285110473633,
  0.1705610752105713,
  0.23955488204956055,
  0.16408133506774902)}

### NMF

In [0]:
# method on the given link :http://surpriselib.com/ and https://surprise.readthedocs.io/en/stable/getting_started.html#use-cross-validation-iterators
nmf = NMF(n_factors = 50)
# Run 5-fold cross-validation and print results.
cross_validate(nmf, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0326  1.0264  1.0309  1.0319  1.0359  1.0315  0.0031  
MAE (testset)     0.7886  0.7847  0.7880  0.7860  0.7913  0.7877  0.0023  
Fit time          9.63    9.63    9.59    9.67    9.51    9.61    0.06    
Test time         0.14    0.14    0.21    0.14    0.21    0.17    0.04    


{'fit_time': (9.634071111679077,
  9.634872198104858,
  9.590462684631348,
  9.672124862670898,
  9.507271766662598),
 'test_mae': array([0.78857141, 0.78474383, 0.78801656, 0.78602596, 0.79129897]),
 'test_rmse': array([1.03258955, 1.02637015, 1.03091883, 1.03186212, 1.03589088]),
 'test_time': (0.13693904876708984,
  0.13797521591186523,
  0.21083378791809082,
  0.13579916954040527,
  0.214555025100708)}

In [0]:
nmf = NMF()

In [0]:
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.25)

# We'll use the famous SVD algorithm.
# sv = SVD(n_factors = 25)

# Train the algorithm on the trainset, and predict ratings for the testset
nmf.fit(trainset)
predictions = nmf.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.9615


0.9615104578929565

### SGD

In [0]:
# # https://surprise.readthedocs.io/en/stable/prediction_algorithms.html
# bsl_options = {'method': 'sgd',
#                'learning_rate': .00005,
#                }
# sgd = BaselineOnly(bsl_options=bsl_options)

# # Run 5-fold cross-validation and print results.
# cross_validate(sgd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0676  1.0832  1.0793  1.0839  1.0778  1.0784  0.0058  
MAE (testset)     0.8942  0.9042  0.9024  0.9066  0.9008  0.9016  0.0042  
Fit time          0.29    0.31    0.33    0.30    0.30    0.31    0.01    
Test time         0.09    0.09    0.09    0.16    0.09    0.10    0.03    


{'fit_time': (0.28616905212402344,
  0.3145027160644531,
  0.32622647285461426,
  0.3026456832885742,
  0.29848599433898926),
 'test_mae': array([0.89420241, 0.90419632, 0.90240537, 0.90659145, 0.90075609]),
 'test_rmse': array([1.06761678, 1.08317121, 1.07931041, 1.08388333, 1.07779745]),
 'test_time': (0.08976030349731445,
  0.08913922309875488,
  0.08933877944946289,
  0.15516257286071777,
  0.08840465545654297)}

In [0]:
# # https://www.ethanrosenthal.com/2016/01/09/explicit-matrix-factorization-sgd-als/
# # Load data from disk
# # names = ['user_id', 'item_id', 'rating']
# # df = pd.read_csv('u.data', sep='\t', names=names)
# n_users = df.userId.unique().shape[0]
# n_items = df.movieId.unique().shape[0]

# # Create r_{ui}, our ratings matrix
# ratings = np.zeros((n_users, n_items))
# for row in df.itertuples():
#     ratings[row[1]-1, row[2]-1] = row[3]

# # Split into training and test sets. 
# # Remove 10 ratings for each user 
# # and assign them to the test set
# def train_test_split(ratings):
#     test = np.zeros(ratings.shape)
#     train = ratings.copy()
#     for user in xrange(ratings.shape[0]):
#         test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
#                                         size=10, 
#                                         replace=False)
#         train[user, test_ratings] = 0.
#         test[user, test_ratings] = ratings[user, test_ratings]
        
#     # Test and training are truly disjoint
#     assert(np.all((train * test) == 0)) 
#     return train, test

# train, test = train_test_split(ratings)

IndexError: ignored

In [0]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)


In [0]:
from numpy.linalg import solve

class ExplicitMF():
    def __init__(self, 
                 ratings, 
                 n_factors=40, 
                 item_reg=0.0, 
                 user_reg=0.0,
                 verbose=False):
        """
        Train a matrix factorization model to predict empty 
        entries in a matrix. The terminology assumes a 
        ratings matrix which is ~ user x item
        
        Params
        ======
        ratings : (ndarray)
            User x Item matrix with corresponding ratings
        
        n_factors : (int)
            Number of latent factors to use in matrix 
            factorization model
        
        item_reg : (float)
            Regularization term for item latent factors
        
        user_reg : (float)
            Regularization term for user latent factors
        
        verbose : (bool)
            Whether or not to printout training progress
        """
        
        self.ratings = ratings
        self.n_users, self.n_items = ratings.shape
        self.n_factors = n_factors
        self.item_reg = item_reg
        self.user_reg = user_reg
        self._v = verbose

    def als_step(self,
                 latent_vectors,
                 fixed_vecs,
                 ratings,
                 _lambda,
                 type='user'):
        """
        One of the two ALS steps. Solve for the latent vectors
        specified by type.
        """
        if type == 'user':
            # Precompute
            YTY = fixed_vecs.T.dot(fixed_vecs)
            lambdaI = np.eye(YTY.shape[0]) * _lambda

            for u in range(latent_vectors.shape[0]):
                latent_vectors[u, :] = solve((YTY + lambdaI), 
                                             ratings[u, :].dot(fixed_vecs))
        elif type == 'item':
            # Precompute
            XTX = fixed_vecs.T.dot(fixed_vecs)
            lambdaI = np.eye(XTX.shape[0]) * _lambda
            
            for i in range(latent_vectors.shape[0]):
                latent_vectors[i, :] = solve((XTX + lambdaI), 
                                             ratings[:, i].T.dot(fixed_vecs))
        return latent_vectors

    def train(self, n_iter=10):
        """ Train model for n_iter iterations from scratch."""
        # initialize latent vectors
        self.user_vecs = np.random.random((self.n_users, self.n_factors))
        self.item_vecs = np.random.random((self.n_items, self.n_factors))
        
        self.partial_train(n_iter)
    
    def partial_train(self, n_iter):
        """ 
        Train model for n_iter iterations. Can be 
        called multiple times for further training.
        """
        ctr = 1
        while ctr <= n_iter:
            if ctr % 10 == 0 and self._v:
                print ('\tcurrent iteration: {}').format(ctr)
            self.user_vecs = self.als_step(self.user_vecs, 
                                           self.item_vecs, 
                                           self.ratings, 
                                           self.user_reg, 
                                           type='user')
            self.item_vecs = self.als_step(self.item_vecs, 
                                           self.user_vecs, 
                                           self.ratings, 
                                           self.item_reg, 
                                           type='item')
            ctr += 1
    
    def predict_all(self):
        """ Predict ratings for every user and item. """
        predictions = np.zeros((self.user_vecs.shape[0], 
                                self.item_vecs.shape[0]))
        for u in range(self.user_vecs.shape[0]):
            for i in range(self.item_vecs.shape[0]):
                predictions[u, i] = self.predict(u, i)
                
        return predictions
    def predict(self, u, i):
        """ Single user and item prediction. """
        return self.user_vecs[u, :].dot(self.item_vecs[i, :].T)
    
    def calculate_learning_curve(self, iter_array, test):
        """
        Keep track of MSE as a function of training iterations.
        
        Params
        ======
        iter_array : (list)
            List of numbers of iterations to train for each step of 
            the learning curve. e.g. [1, 5, 10, 20]
        test : (2D ndarray)
            Testing dataset (assumed to be user x item).
        
        The function creates two new class attributes:
        
        train_mse : (list)
            Training data MSE values for each value of iter_array
        test_mse : (list)
            Test data MSE values for each value of iter_array
        """
        iter_array.sort()
        self.train_mse =[]
        self.test_mse = []
        iter_diff = 0
        for (i, n_iter) in enumerate(iter_array):
            if self._v:
                print ('Iteration: {}').format(n_iter)
            if i == 0:
                self.train(n_iter - iter_diff)
            else:
                self.partial_train(n_iter - iter_diff)

            predictions = self.predict_all()

            self.train_mse += [get_mse(predictions, self.ratings)]
            self.test_mse += [get_mse(predictions, test)]
            if self._v:
                print ('Train mse: ' + str(self.train_mse[-1]))
                print ('Test mse: ' + str(self.test_mse[-1]))
            iter_diff = n_iter


In [0]:
MF_ALS = ExplicitMF(train_data, n_factors=40, \
                    user_reg=0.0, item_reg=0.0)
iter_array = [1, 2, 5, 10, 25, 50, 100]
MF_ALS.calculate_learning_curve(iter_array, test_data)

TypeError: ignored